# 教師あり多クラス分類のノートブック

In [1]:
from matplotlib import pyplot as plt
import polars as pl
import seaborn as sns
import japanize_matplotlib

from common import *

In [2]:
# ハイパーパラメータ
N_COMPONENTS = 50
TEST_SIZE = 0.2
VAL_SIZE = 0.2

In [3]:
# データセットの読み込み
X_train, X_val, X_test, y_train, y_val, y_test = load_mnist_pca_train_test_val(
    n_components=N_COMPONENTS, test_size=TEST_SIZE, val_size=VAL_SIZE
)

Files already downloaded
Files already downloaded


In [4]:
# LightGBMの学習用の関数
def train_lgbm_with_custom_loss(X_train, X_valid, y_train, y_valid):

    """Train LightGBM with custom loss function."""
    train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
    valid_data = lgb.Dataset(X_valid, label=y_valid, free_raw_data=False)

    params = {
        "objective": "multiclass",
        "verbose": -1,
        "learning_rate": 0.1,
        "num_boost_round": 200,
        "num_class": 10,
    }
    valid_accuracies = []

    def record_accuracies(p: lgb.Booster, train_data: lgb.Dataset, valid_data: lgb.Dataset):
        valid_pred = p.predict(valid_data.data).argmax(-1)
        valid_acc = accuracy_score(valid_data.label, valid_pred)
        valid_accuracies.append(valid_acc)
        print(valid_acc)
    
    gbm = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        callbacks=[lambda p: record_accuracies(p.model, train_data, valid_data)]
    )
    return gbm, valid_accuracies

In [5]:
gbm, valid_accuracies =train_lgbm_with_custom_loss(X_train, X_val, y_train, y_val)
y_pred = gbm.predict(X_val)
print("Accuracy: {}".format(accuracy_score(y_val, y_pred.argmax(-1))))

/Users/ryoma.kobayashi/gits/techblog_git/wsl_w_lgbm/.venv/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


0.75
0.8221666666666667
0.8424166666666667
0.8539166666666667
0.8615
0.8679166666666667
0.87175
0.8751666666666666
0.8785833333333334
0.8823333333333333
0.88675
0.8894166666666666
0.89325
0.8955833333333333
0.8995
0.9023333333333333
0.906
0.9080833333333334
0.9105833333333333
0.91375
0.9159166666666667
0.9169166666666667
0.9191666666666667
0.9208333333333333
0.9231666666666667
0.9248333333333333
0.9259166666666667
0.9274166666666667
0.9284166666666667
0.9305
0.9315833333333333
0.9339166666666666
0.9345
0.9364166666666667
0.9365833333333333
0.937
0.9383333333333334
0.9396666666666667
0.9415
0.9413333333333334
0.9418333333333333
0.9423333333333334
0.9435833333333333
0.9440833333333334
0.94475
0.9449166666666666
0.9459166666666666
0.946
0.9465833333333333
0.947
0.9478333333333333
0.9486666666666667
0.94875
0.94925
0.95025
0.9505
0.9513333333333334
0.9515833333333333
0.952
0.9529166666666666
0.9529166666666666
0.95425
0.9546666666666667
0.9545833333333333
0.9555
0.9555833333333333
0.955666

In [6]:
np.save("sup_multi_valid_accuracies.npy", valid_accuracies)